<a href="https://colab.research.google.com/github/MaxManfred/deep-learning/blob/master/notebooks/Movie_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indroduction

**CLASSIFYING MOVIE REVIEWS: A BINARY CLASSIFICATION EXAMPLE**

In this example, you’ll learn to classify movie reviews as positive or negative, based on the text content of the reviews.

You’ll work with the IMDB dataset: a set of 50,000 highly polarized reviews from the Internet Movie Database. 
They’re split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews.

Just like the MNIST dataset, the IMDB dataset comes packaged with Keras. It has already been preprocessed: the reviews (sequences of words) have been turned into sequences of integers, where each integer stands for a specific word in a dictionary.

In [0]:
# import needed libraries
from keras.datasets import imdb
from keras import models
from keras import layers
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt

# Data preparation

Load IMDB data set in Keras.

The argument num_words=10000 means you’ll only keep the top 10,000 most  frequently occurring words in the training data. 

Rare words will be discarded. 

This allows you to work with vector data of manageable size.

The variables train_data and test_data are lists of reviews; each review is a list of word indices (encoding a sequence of words). 
train_labels and test_labels are lists of 0s and 1s, where 0 stands for negative and 1 stands for positive: bold text

In [0]:
(training_samples, training_labels), (test_samples, test_labels) = imdb.load_data(num_words=10000)

We print the first sample review in the training set and its label.

We extract word_index from IMDB: word_index is a dictionary mapping words to an integer index.

We then create reverse_word_index which reverses word_index, mapping integer indices to words.

Note that the indices are offset by 3 because 0, 1, and 2 are reserved indices for “padding,” “start of sequence,” and “unknown.”.

Finally we print the decoded review.

In [0]:
# Because you’re restricting yourself to the top 10,000 most frequent words, no 
# word index will exceed 10,000
print('Highest index of selected terms in the reviews: ', 
      max( [max(review) for review in training_samples] ))

# as an example, show a sample
print('Training sample: ', training_samples[0])
print('Training label: ', training_labels[0])

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') + '(' + str(i - 3) + ')' for i in training_samples[0]])
print('Decoded training sample: ', decoded_review)

You can’t feed lists of integers into a neural network. 

You have to turn your lists into tensors. 

One-hot encode your lists to turn them into vectors of 0s and 1s. 

This would mean, for instance, turning the sequence [3, 5] into a 10,000-dimensional vector that would be all 0s except for indices 3 and 5, which would be 1s. 

Then you could use as the first layer in your network a Dense layer, capable of handling floating-point vector data.

In [0]:
# define the vectorization process as a function
def vectorize_reviews(reviews, dimension = 10000):
    results = np.zeros((len(reviews), dimension))
    for i, review in enumerate(reviews):
        results[i, review] = 1.
    return results
  
# vectorize samples
vectorized_training_samples = vectorize_reviews(training_samples)
vectorized_test_samples = vectorize_reviews(test_samples)

# vectorize labels
vectorized_training_labels = np.asarray(training_labels).astype('float32')
vectorized_test_labels = np.asarray(test_labels).astype('float32')

In [0]:
# print a vectorized sample
# print(vectorized_training_samples[0])

# create a set from the first review and sort it in ascending order to print the 
# indices of the terms to compare them with the vectorization
first_sample = list(set(list.copy(training_samples[0])))
first_sample.sort()
print('First 100 sorted term indices in first training sample: ', first_sample[0:100])
print('First 100 coordinated of the vectorization of the first training sample: ', vectorized_training_samples[0][0:100,])

The input data is vectors, and the labels are scalars (1s and 0s): this is the easiest setup you’ll ever encounter. 

A type of network that performs well on such a problem is a simple stack of fully connected (Dense) layers with relu activations: Dense(16, activation='relu').

The argument being passed to each Dense layer (16) is the number of hidden units of the layer. 

A hidden unit is a dimension in the representation space of the layer.
Each such Dense layer with a relu activation implements the following chain of tensor operations:

`output = relu(dot(W, input) + b)`

Having 16 hidden units means the weight matrix W will have shape (input_dimension, 16): the dot product with W will project the input data onto a 16-dimensional representation space (and then you’ll add the bias vector b and apply the relu operation). 

You can intuitively understand the dimensionality of your representation space as “how much freedom you’re allowing the network to have when learning internal representations.” 

Having more hidden units (a higher-dimensional representation space) allows your network to learn more-complex representations, but it makes the network more computationally expensive and may lead to learning unwanted patterns (patterns that will improve performance on the training data but not on the test data).

There are two key architecture decisions to be made about such a stack of Dense layers:

*   How many layers to use
*   How many hidden units to choose for each layer

Later on you’ll learn formal principles to guide you in making these choices. 
For the time being, let's use the following architecture choice:

*   Two intermediate layers with 16 hidden units each
*   A third layer that will output the scalar prediction regarding the sentiment of the current review

The intermediate layers will use **relu** as their activation function, and the final layer will use a **sigmoid** activation so as to output a probability (a score between 0 and 1, indicating how likely the sample is to have the target “1”: how likely the review is to be positive).

Finally, you need to choose a loss function and an optimizer. 

Because you’re facing a binary classification problem and the output of your network is a probability (you end your network with a single-unit layer with a sigmoid activation), 

it’s best to use the **binary_crossentropy** loss. It isn’t the only viable choice: you could use, for instance, mean_squared_error. But crossentropy is usually the best choice when you’re dealing with models that output probabilities.

# Neural network topology and optimization

In [0]:
# create neural network
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
# define optimization process
model.compile(
    optimizer = optimizers.RMSprop(lr = 0.001), 
    loss = 'binary_crossentropy', 
    metrics = ['accuracy']
)

# Neural network training with validation

In order to monitor during training the accuracy of the model on data it has never seen before, you’ll create a validation set by setting apart 10,000 samples from the original training data.


In [0]:
# slice the sample and labels training data set to accomodate room for a validation
# data set
vectorized_validation_samples = vectorized_training_samples[:10000]
vectorized_partial_training_samples = vectorized_training_samples[10000:]

vectorized_validation_labels = vectorized_training_labels[:10000]
vectorized_partial_training_labels = vectorized_training_labels[10000:]

You’ll now train the model for 20 epochs (20 iterations over all samples), in mini-batches of 512 samples. 
At the same time, you’ll monitor loss and accuracy on the 10,000 samples that you set apart. 
You do so by passing the validation data as the validation_data argument.

In [0]:
history = model.fit(
    vectorized_partial_training_samples,
    vectorized_partial_training_labels,
    epochs = 20, batch_size = 512,
    validation_data=(vectorized_validation_samples, vectorized_validation_labels)
)

At the end of every epoch, there is a slight pause as the model computes its loss and accuracy on the 10,000 samples of the validation data.

Note that the call to model.fit() returns a History object. This object has a member history, which is a dictionary containing data about everything that happened during training.

The dictionary contains four entries: one per metric that was being monitored during training and during validation. 

In [0]:
history_dict = history.history
print('History keys: ', history_dict.keys())
print('\n')

# display loss using matplot lib
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(history_dict['acc']) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# clear diagram
plt.clf()
print('\n')

# display accuracy using matplotlib
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

As you can see, the training loss decreases with every epoch, and the training accuracy increases with every epoch. 

That’s what you would expect when running gradient-descent optimization—the quantity you’re trying to minimize should be less with every iteration. 

But that isn’t the case for the validation loss and accuracy: they seem to peak at the fourth epoch. 

This is an example of what we warned against earlier: a model that performs better on the training data isn’t necessarily a model that will do better on data it has never seen before. 

In precise terms, what you’re seeing is **overfitting**: after the second epoch, you’re overoptimizing on the training data, and you end up learning representations that are specific to the training data and don’t generalize to data outside of the training set.

In this case, to prevent overfitting, you could stop training after three epochs. In general, you can use a range of techniques to mitigate overfitting.

Let’s train a new network from scratch for four epochs and then evaluate it on the test data (loss and accuracy).

This fairly naive approach achieves an accuracy of 88%. With state-of-the-art approaches, you should be able to get close to 95%

# Neural network training and evaluation

In [0]:
model.fit(vectorized_training_samples, vectorized_training_labels, epochs = 4, batch_size = 512)
results = model.evaluate(vectorized_test_samples, vectorized_test_labels)
print(results)

After having trained a network, you’ll want to use it in a practical setting. 

You can generate the likelihood of reviews being positive by using the predict method.

As you can see, the network is confident for some samples (0.99 or more, or 0.01 or less) but less confident for others (0.6, 0.4).

In [0]:
model.predict(vectorized_test_samples)

# Further experiments

The following experiments will help convince you that the architecture choices you’ve made are all fairly reasonable, although there’s still room for improvement:

1.   You used two hidden layers. Try using one or three hidden layers, and see how doing so affects validation and test accuracy.
2.   Try using layers with more hidden units or fewer hidden units: 32 units, 64 units, and so on.
3.   Try using the **mse** loss function instead of **binary_crossentropy**.
4.   Try using the **tanh** activation (an activation that was popular in the early days of neural networks) instead of **relu**.



# Wrapping up

Here’s what you should take away from this example:

*   You usually need to do quite a bit of preprocessing on your raw data in order to be able to feed it—as tensors—into a neural network. Sequences of words can be encoded as binary vectors, but there are other encoding options, too.
*   Stacks of Dense layers with relu activations can solve a wide range of problems (including sentiment classification), and you’ll likely use them frequently.
*   In a binary classification problem (two output classes), your network should end with a Dense layer with one unit and a sigmoid activation: the output of your network should be a scalar between 0 and 1, encoding a probability.
*   With such a scalar sigmoid output on a binary classification problem, the loss function you should use is binary_crossentropy.
* .  The rmsprop optimizer is generally a good enough choice, whatever your problem. That’s one less thing for you to worry about.
* .  As they get better on their training data, neural networks eventually start overfitting and end up obtaining increasingly worse results on data they’ve never seen before. Be sure to always monitor performance on data that is outside of the training set.